In [7]:
from bs4 import BeautifulSoup
import requests
import time

#ヤフーニュースの情報を取得
def yahoo_news_scraping(topic):
    url = 'https://news.yahoo.co.jp/'
    r = requests.get(url)
    soup = BeautifulSoup(r.text, "html.parser")
    
    list_item = soup.find_all('li')
    #もしかしたらエラーの原因になるかも
    list_item = list_item[12:21]
    
    # <a>要素からトピックを取得
    a_contents = [li.a for li in list_item]
    
    articles_dict = []
    topic = topic 
    #トピックのURL取得
    for i, a_content in enumerate(a_contents):
        if topic in a_content.text:
            categories_num = i
    
    a_content = a_contents[categories_num]
    
    topic_url = a_content['href']
    URL = url + topic_url
    
    r = requests.get(URL)
    soup = BeautifulSoup(r.text, "html.parser")
    #クラス名が変わるので注意！！
    elements = soup.select('.sc-fXazdy.UjHkE')
    
    elements = elements[:5]
    
    news_urls = []
    news_titles = []
    #ニュースのURLとタイトルを取得
    for element in elements:
        url = element.a['href']
        title = element.text[1:]
        news_urls.append(url)
        news_titles.append(title)
            
    news_text = []
    #ニュースの本文を取得
    for news_url in news_urls:
        r = requests.get(news_url)
        soup = BeautifulSoup(r.text, "html.parser")
        #クラス名が変わるので注意！！ 
        elements = soup.select('.sc-iMCRTP.ePfheF.yjSlinkDirectlink.highLightSearchTarget')
    
        text = ""
    
        #ニュース記事の本文が配列で区切られていた場合
        for i in range(0, 8):
            try:
                if (i == 0):
                    text = elements[0].text
                else:
                    text += elements[i].text
            except:
                pass
    
        #ページが1つだけのとき、エラーが出るため
        try:
            #クラス名が変わるので注意！！ （取得例：２ページ)
            page_element = soup.select('.sc-cpUASM.bqNXll')
            page = page_element[0].text         
            page_str = ''.join(filter(str.isdigit, page))
            page_num = int(page_str)
        except:
            page_num = 1
            
        #ページ数が複数のとき、ページ遷移し本文を取得
        if (page_num > 1):
            for i in range(2, page_num + 1):
                news_url_page = news_url + "?page=" + str(page_num)
                r = requests.get(news_url_page)
                soup = BeautifulSoup(r.text, "html.parser")
                #クラス名が変わるので注意！！ 
                elements = soup.select('.sc-iMCRTP.ePfheF.yjSlinkDirectlink.highLightSearchTargett')
                #ニュース記事の本文が配列で区切られていた場合
                for i in range(0, 8):
                    try:
                        text += elements[i].text
                    except:
                        pass
                        
        news_text.append(text)
    
        time.sleep(0.5)

    articles = []
    for url, title, text in zip(news_urls, news_titles, news_text):
        article = {
            "title" : title,
            "URL" : url,
            "text" : text
        }
        articles.append(article)

    return articles

#CNNニュースの情報を取得
def cnn_news_scraping(topic):
    url = 'https://www.cnn.co.jp'
    r = requests.get(url)
    soup = BeautifulSoup(r.text, "html.parser")
    
    #クラス名が変わるので注意！！
    elements = soup.select('.pg-container li a')
    elements = elements[6:11] + [elements[14], elements[16]]
    
    topic = topic
    #トピックのURL取得
    for i, element in enumerate(elements):
        if topic in element.text:
            categories_num = i
    
    element = elements[categories_num]
    
    topic_url = element['href']
    URL = url + topic_url
    
    r = requests.get(URL)
    soup = BeautifulSoup(r.text, "html.parser")
    
    #クラス名が変わるので注意！！
    elements = soup.select('.cd a')
    
    elements = elements[:3]
    
    news_urls = []
    news_titles = []
    #ニュースのURLとタイトルを取得
    for element in elements:
        news_url =  url + element['href']
        news_title = element.text
    
        news_urls.append(news_url)
        news_titles.append(news_title)
    
    news_text = []
    #ニュースの本文を取得
    for i, news_url in enumerate(news_urls):
        text = ""
        
        for j in range(1, 7):
            num = str(j)
            page = '-' + num + '.html'    
            # 文字列の置換を行い新しいURLを生成
            news_url_page = news_urls[i].replace('.html', page)
    
            #ページ数が複数のとき、ページ遷移し本文を取得
            response = requests.get(news_url_page)
    
            # レスポンスのステータスコードを確認
            if response.status_code != 404: 
                r = requests.get(news_url_page)
                soup = BeautifulSoup(r.text, "html.parser")
                #クラス名が変わるので注意！！ 
                elements = soup.find(id = 'leaf-body')
                text += elements.text
            else:
                continue
    
        news_text.append(text)

    articles = []
    for url, title, text in zip(news_urls, news_titles, news_text):
        article = {
            "title" : title,
            "URL" : url,
            "text" : text
        }
        articles.append(article)

    return articles

#データを整える
def clean_data(articles):
    for i in range(len(articles)):
        articles[i]['title'] = articles[i]['title'].replace('\u3000', '') 
        articles[i]['title'] = articles[i]['title'].lstrip('0123456789')
        articles[i]['text'] = articles[i]['text'].replace('\u3000', '')
        articles[i]['text'] = articles[i]['text'].replace('\n', '')

    return articles


In [9]:
articles_dict = yahoo_news_scraping('経済')
articles = clean_data(articles_dict)

In [10]:
articles

[{'title': '「雨の日に洗車」なぜ勧める？ ガソリンスタンドで「推奨」するワケ… 実は「雨で汚れ」は落ちない？くるまのニュース10/4(水)14:10',
  'URL': 'https://news.yahoo.co.jp/articles/2e0cb7020e99449874ea81b6298aa3567dbdb13c',
  'text': '雨が降ってくると「クルマが汚れているからちょうどいい」と言う人がいます。一方でガソリンスタンドなどでは「雨の日洗車キャンペーン」や「雨の日は洗車がオトク！」といった宣伝をしていますが、なぜ雨の日に洗車を勧めてくるのでしょうか。【衝撃画像！】「えっ…！」これが危険すぎる「タイヤの末路」です（17枚）クルマの洗車は、こだわる人であれば自らの手で洗車することもあり、様々な洗車グッズも展開されています。一方で、自分でやるのが面倒と考える人もおり、その場合は汚れが目立った際に洗車機で洗うという人も一定数存在します。どちらの場合でも、雨の日にであれば「雨水が汚れを洗い流してくれるので洗車は必要ない」と考える人もいるかもしれません。クルマは新車時であれば、塗装やワックスなどがボディを守ってくれています。しかし時間が経つにつれ、ボディを守る力が弱まり、ホコリやゴミが付着しやすくなってしまいます。そして、そのホコリやゴミなどが酸化することで頑固な汚れの原因となるのです。また、小さな虫の死骸もボディにとって害となり、虫の死骸にはタンパク質やリン酸が含まれており、塗装面に悪影響を及ぼし「酸化クレーター」と呼ばれる凸凹が生じることも。さらに雨には大気中のチリやゴミといった汚れが含まれており、そのまま放置した場合、水垢やウォータースポットになります。ほかにも、ボディに付着した砂や鉄は、雨に濡れると泥団子状の塊となり、濡れている間は柔らかいものの、雨がやんで乾くと硬くなり、ボディを傷付ける原因となるようです。こうしたことをふまえてクルマの洗車について洗車専門店では次のように話しています。「基本的には、1、2か月程度の頻度で洗車をおこなっていただくことをおすすめしています。また雨にあたることでクルマのボディに傷がつきやすい状態になってしまいます。そのため『雨で汚れが落ちるだろう』という考えから洗車を怠っていた場合、塗装や誇りなどのごみの付着